This is the pracctice 1 session of debiasing Facial Recognitation

In [1]:
#1 Imports
import os
import random
from pathlib import Path
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt 

import torch
import mitdeeplearning as mdl


In [2]:
# Dataset Download
CACHE_DIR = Path.cwd() /".cache"/"mitdeeplearning"
CACHE_DIR.mkdir (parents= True, exist_ok= True)

path_to_training_data = CACHE_DIR.joinpath("train_face.h5")

#Check to avoid re-downloading
if path_to_training_data.is_file():
    print("Data already exists")
else:
    url = "https://www.dropbox.com/s/hlz8atheyozp1yx/train_face.h5?dl=1"
    torch.hub.download_url_to_file(url,path_to_training_data)
    print("Dowloading data to: {path_to_training_data} ")



Data already exists


In [3]:
# 3A. DataLoader

channels_last = False
loader = mdl.lab2.TrainingDatasetLoader(path_to_training_data,channels_last=channels_last)

images, labels = loader.get_batch(100)


Opening d:\DL_Repos\introtodeeplearning\HayatLabs\.cache\mitdeeplearning\train_face.h5
Loading data into memory...


In [4]:
#3B Checking data size and dimensions

dataset_size = loader.get_train_size()
print(f"Size of training dataset: {dataset_size}")

B,C,H,W = images.shape
print(f"Bach_size:{B}, Channels: {C}, Height:{H}, Width:{W}")


Size of training dataset: 109914
Bach_size:100, Channels: 3, Height:64, Width:64


In [5]:
print(f"Shape of Labels: {labels.shape} Shows that it is a {labels.ndim}-D Array having Rows: {labels.shape[0]} and Columns: {labels.shape[1]}") 

lab = np.where(labels==1)
faces = lab[0]
no_faces = lab[1]
print(f"Faces (Where label =1) : {lab}") # it provided a tupal of arrays

print(f"Selecting First array (Faces): {faces}") #selecting the first element of the array
print(f"Selecting 2nd Array (non-faces):{no_faces} ")


Shape of Labels: (100, 1) Shows that it is a 2-D Array having Rows: 100 and Columns: 1
Faces (Where label =1) : (array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]))
Selecting First array (Faces): [50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
Selecting 2nd Array (non-faces):[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0] 


# 4 Display of Dataset

In [6]:
def show_images(idx_face=0, idx_notface = 0):
    face_img = images[faces].transpose(0,2,3,1)
    notface_img = images[no_faces].transpose(0,2,3,1)

    plt.figure(figsize=(5,5))
    
    plt.subplot(1,2,1)
    plt.imshow(face_img[idx_face])
    plt.title("faces")
    plt.grid(False)
    
    plt.subplot(1,2,2)
    plt.imshow(notface_img[idx_notface])
    plt.title("Not Face")

    plt.show()
    

widgets.interact(show_images, idx_face=(0,49), idx_notface=(0,49))



interactive(children=(IntSlider(value=0, description='idx_face', max=49), IntSlider(value=0, description='idx_…

<function __main__.show_images(idx_face=0, idx_notface=0)>